<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
موضوع‌بندی
</font>
</h1>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
به یک تمرین صنعتی از  کاربرد یادگیری ماشین در پردازش زبان طبیعی (NLP) خوش آمدید.  در این تمرین داده‌های واقعی وب فارسی که توسط پلتفرم <a href="https://www.yektanet.com/">یکتانت</a> پالایش و جمع‌آوری شده در اختیار ما قرار گرفته است. هدف تمرین؛ ساخت یک مدل یادگیری ماشین است که با توجه متن‌های موجود در یک پیوند (Link) نظیر <i>عنوان</i>، <i>توضیحات</i>، <i>محتوای متنی [کامل]</i> و غیره بتواند دسته‌ی موضوعی آن سند را پیش‌بینی کند. به‌عنوان مثال اگر پیوندی از یک سایت خبری با عنوان «<i>کیهان کلهر جایزه مرد سال موسیقی جهان را دریافت کرد</i>» داشته باشیم، مدل ما باید پیش‌بینی کند که این مطلب مرتبط با موضوع «موسیقی» است. البته در این مثال ما تنها از ویژگی <i>عنوان</i> یاد کردیم، در حالی‌که می‌توان از متن <i>توضیحات</i> یا <i>محتوای متنی</i> هم کمک گرفت. 

</br>
توجه داشته باشید برای آن‌که بتوانید از الگوریتم‌های معرفی‌شده در کالج جهت کار روی داده‌های متنی استفاده کنید نیاز است حداقل با یکی از روش‌های تعبیه‌سازی (Embedding) آشنا شده باشید تا بتوانید هر متن را به یک بردار ویژگی عددی تبدیل کنید.

</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
وارد کردن کتابخانه‌های مورد نیاز
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    ابتدا کتابخانه‌های مورد نیازتان را وارد کنید.
</font>
</p>

In [4]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
import scipy.sparse as sp

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
معرفی مجموعه‌داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    هر نمونه از این مجموعه‌داده با ویژگی‌هایی که در جدول زیر شرح داده شده همراه است. ستون <code>category</code> متغیر هدف مسئله است که موضوع محتوا را نشان می‌دهد. 
</font>
</p>
<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>category</code>| موضوع (متغیر هدف) |
|<code>description</code>| توضیحات |
|<code>text_content</code>| محتوای متنی |
|<code>title</code>| عنوان |
|<code>h1</code>| محتوای تگ <code>h1</code> صفحه |
|<code>h2</code>|محتوای تگ <code>h2</code> صفحه  |
|<code>url</code>| آدرس پیوند|
|<code>domain</code>|دامنه‌ی وب‌سایت |
|<code>id</code>|آیدی پیوند|

</font>
</div>
</center>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    در ابتدا نیاز است فایل‌های مجموعه‌داده را بخوانید. نمونه‌های آموزشی در فایل <code>yektanet_train.csv</code> و نمونه‌های آزمون که باید موضوع آن‌ها را پیش‌بینی کنید در فایل <code>yektanet_test.csv</code> ذخیره شده‌اند. اگر لازم دانستید می‌توانید به دلخواه خود بخشی از مجموعه‌ی آموزشی را به عنوان مجموعه‌ی اعتبارسنجی نیز جدا کنید.
</font>
</p>

In [2]:
train = pd.read_csv('../data/yektanet_train.csv')
train.head()

,category,description,text_content,title,h1,h2,url,domain,id
0,کتاب و ادبیات,"از شوبنده ها: جستجو معنی ""از شوبنده ها"" در فره...",معنی از شوبنده ها | جدول یاب از شوبنده ها 381,معنی از شوبنده ها | جدول یاب,معنی از شوبنده ها,از شوبنده ها در معادل ابجد,jadvalyab.ir/search?q=%D8%A7%D8%B2+%D8%B4%D9%8...,jadvalyab.ir,158
1,تجارت و اقتصاد,بیت‌کوین کش یک ارز مجازی مشهور است و بیت‌کوین ...,عکس بیت‌کوین کش برای پروفایل عکس و والپیپرهای ...,عکس بیت‌کوین کش برای پروفایل,عکس بیت‌کوین کش برای پروفایل,عکس بیت کوین با کیفیت 4K عکس ارزهای دیجیتال عک...,jowhareh.com/photo/%D8%B9%DA%A9%D8%B3-%D8%A8%D...,jowhareh.com,3268
2,سلامت,نوبت دهی دکتر مهناز عابدینی متخصص رادیولوژی و ...,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...,دکتر مهناز عابدینی,آدرس و تلفن دکتر مهناز عابدینی نظرات و تجربیات...,doctor-yab.ir/Search/14773/%D8%AF%DA%A9%D8%AA%...,doctor-yab.ir,175
3,تکنولوژی و کامپبوتر,نرم افزار Geph برای اندروید یک پلت‌فرم چندسکوی...,دانلود تحریم‌گذر Geph برای اندروید خانه/اندروی...,دانلود تحریم‌گذر Geph برای اندروید,دانلود تحریم‌گذر Geph برای اندروید,دانلود نرم افزار Geph,palexe.site/dl/geph-android/,palexe.site,3402
4,تکنولوژی و کامپبوتر,سری جدید تلویزیون‌های هوشمند سامسونگ که با نام...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ,راه‌اندازی تلویزیون همگام‌سازی کنترل اتصال به ...,rokhdadeghtesadi.ir/43874/,rokhdadeghtesadi.ir,3811


In [3]:
test = pd.read_csv('../data/yektanet_test.csv')
test.head()

,description,text_content,title,h1,h2,url,domain,id
0,پیام ناشناس یک ابزار رایگان برای ساخت چالش های...,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,NaN,پیام ناشناس / چالش دریافت پیام های ناشناس با ل...,abzarek.ir/service-p/msg/824757/,abzarek.ir,3291
1,اقتصادنیوز: سید امیر احمد علیه السلام ملقب به ...,حرم شاهچراغ شیراز در گذر زمان+تصاویر حرم شاهچر...,حرم شاهچراغ شیراز در گذر زمان+تصاویر,حرم شاهچراغ شیراز در گذر زمان+تصاویر,NaN,eghtesadnews.com/%D8%A8%D8%AE%D8%B4-%D8%A7%D8%...,eghtesadnews.com,5053
2,آخرین خبر های بین المللی و آخرین تحولات جهان، ...,آخرین خبر | زلنسکی پیام پوتین را دریافت کرد ✖...,آخرین خبر | زلنسکی پیام پوتین را دریافت کرد,زلنسکی پیام پوتین را دریافت کرد,NaN,akharinkhabar.ir/world/9332333/%D8%B2%D9%84%D9...,akharinkhabar.ir,564
3,آهنگ گاهی دریدن لازم نیست با صدای امیر تتلو با...,آهنگ گاهی دریدن لازم نیست امیر تتلو آهنگ گاهی ...,آهنگ گاهی دریدن لازم نیست امیر تتلو,آهنگ گاهی دریدن لازم نیست امیر تتلو,گاهی دریدن لازم نیست امیر تتلو,behmusic.com/music/%DA%AF%D8%A7%D9%87%DB%8C-%D...,behmusic.com,3607
4,کیهان با اشاره به حوادث و ناآرامیهای اخیر کشور...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,انتقاد کیهان از امام جمعه اهل سنت زاهدان/ با ا...,برچسب‌ها نظر شما آخرین اخبار سیاسی ایران در بح...,khabaronline.ir/news/1688132/%D8%A7%D9%86%D8%A...,khabaronline.ir,2358


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
در هنگام کار با داده‌های متنی، پیش‌پردازش داده‌ها به کمک تکنیک‌های پردازش زبان طبیعی یکی از مراحل بسیار تاثیرگذار در یادگیری مدل و عملکرد نهایی است.
در تمرین «کامنت‌کاوی» فصل دسته‌بندی با چندین تکنیک رایج پیش‌پردازش خصوصاً برای زبان فارسی آشنا شدید. در این قسمت نیز می‌توانید تابعی بنویسید که یک رشته (<code>string</code>) را گرفته، اصلاحات موردنظر شما رو روی متن اعمال کرده و در نهایت نتیجه را با فرمت یک رشته (<code>string</code>) خروجی دهد. بررسی و تحلیل کلمات موجود در مجموعه‌داده از نظر تعداد رخداد نیز می‌تواند شما را در پیش‌پردازش بهتر یاری کند.
</font>
</p>


In [5]:
PERSIAN_STOPWORDS = set([
    'است', 'که', 'این', 'را', 'با', 'از', 'در', 'به', 'و', 'یا',
    'هم', 'اما', 'تا', 'اگر', 'برای', 'آن', 'ما', 'شما', 'آنها',
    'او', 'من', 'ها', 'های', 'می', 'بود', 'شد', 'شده', 'کرد',
    'داشت', 'داشته', 'دارد', 'دارند', 'باید', 'نیز', 'هر', 'هیچ',
    'همه', 'بعد', 'قبل', 'بین', 'روی', 'زیر', 'بالا', 'پایین',
    'مثل', 'مانند', 'حتی', 'فقط', 'هنوز', 'دیگر', 'بسیار',
    'خیلی', 'خود', 'وی', 'آیا', 'چه', 'چون', 'چرا', 'کجا',
    'کی', 'کدام', 'چند', 'چندین', 'نه', 'نمی', 'بی', 'غیر',
    'جز', 'الا', 'مگر', 'گر', 'اگرچه', 'هرچند', 'پس', 'پیش',
    'لذا', 'چنین', 'چنان', 'همان', 'همین', 'آنکه', 'كه', 'اين',
    'آن', 'ان', 'هايي', 'تمام', 'كل', 'جميع', 'ديگر', 'بر',
    'تر', 'ترين', 'ترین', 'شان', 'شون', 'هستند', 'هستم', 'هستي',
    'نيست', 'نیست', 'نباشد', 'بشود', 'میشود', 'میکند', 'میکنند',
    'كرد', 'كند', 'نكرد', 'نداشت', 'يك', 'یک', 'دو', 'سه',
    'چهار', 'پنج', 'شش', 'هفت', 'هشت', 'نه', 'ده', 'صد', 'هزار'
])

def clean_text(text):

    if pd.isna(text) or not isinstance(text, str):
        return ''
    text = text.replace('ي', 'ی').replace('ك', 'ک').replace('ة', 'ت')
    text = text.replace('\u200c', ' ') 
    text = re.sub(r'[^\u0600-\u06FF\uFB50-\uFDFF\uFE70-\uFEFFa-zA-Z\s]', ' ', text)

    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text


def persian_tokenizer(text):

    tokens = clean_text(text).split()
    tokens = [t for t in tokens if len(t) > 1 and t not in PERSIAN_STOPWORDS]
    return tokens


def build_combined_text(df):

    texts = []
    for _, row in df.iterrows():
        # Weight: title×3, h1×2, description×2, h2×1, text_content×1
        parts = [
            str(row.get('title', '') or '') * 1,         # already repeated in concat
            str(row.get('title', '') or ''),              # ×2 total
            str(row.get('title', '') or ''),              # ×3 total
            str(row.get('h1', '') or ''),
            str(row.get('h1', '') or ''),                 # h1 ×2
            str(row.get('description', '') or ''),
            str(row.get('description', '') or ''),        # desc ×2
            str(row.get('h2', '') or ''),
            str(row.get('text_content', '') or ''),
        ]
        combined = ' '.join(p for p in parts if p.strip())
        texts.append(combined)
    return texts

X_train_text = build_combined_text(train)

X_test_text = build_combined_text(test)

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
در هنگام تعبیه‌سازی به کمک کتابخانه‌ی <code>scikit-learn</code>
از آنجا که ممکن است توکن‌ساز (Tokenizer) مورد استفاده در این کتابخانه مناسب زبان فارسی نباشد، بهتر است از توکن‌سازهای مناسب این زبان استفاده کنیم. کافیست تابعی بنویسید که یک رشته (<code>string</code>) را گرفته و به کمک کتابخانه‌ی موردنظر شما (مثل <code>word_tokenize</code> در کتابخانه‌ی هضم) توکن‌های آن را جدا کند. خروجی تابع لیستی از توکن‌ها خواهد بود.
</font>
</p>


In [7]:
def tokenizer(text):
    return persian_tokenizer(text)

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
توجه داشته باشید انتخاب این‌که از کدام ویژگی‌ها (متن‌ها) به‌عنوان ورودی الگوریتم استفاده کنید بر عهده‌ی خودتان است. می‌توانید تنها از یک ستون مثل <code>title</code> استفاده کنید یا می‌توانید به‌نحوی متن یا حتی بردار ویژگی هر ستون را با همدیگر ترکیب کنید. همچنین فراموش نکنید که ستون متغیر هدف یعنی <code>category</code> نیاز به کدگذاری دارد.
</font>
</p>


In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(train['category'])

for i, cat in enumerate(le.classes_):
    count = (train['category'] == cat).sum()

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_text, y_train,
    test_size=0.15, random_state=42, stratify=y_train
)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
مدل‌سازی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
اکنون وقت آن رسیده که الگوریتم یادگیری ماشین موردنظر خود را بر روی داده‌های آموزشی اجرا کنید. در انتخاب الگوریتم کاملاً آزاد هستید. برای این بخش می‌توانید ابتدا هر ورودی متن را به کمک تکنیک‌های معرفی‌شده در درسنامه‌های این فصل (مثل Bag-of-Word یا Tf-idf) به بردارهای ویژگی عددی تبدیل کنید. سپس این بردارها را همراه با لیست برچسب‌های صحیح به الگوریتم بدهید تا مدل آموخته شود. علاوه بر این می‌توانید تمام این مراحل را در یک خط لوله‌ (<code>Pipeline</code>) نیز تعریف کنید.
</font>
</p>

In [9]:
tfidf_params = dict(
    tokenizer=tokenizer,
    ngram_range=(1, 2),    
    max_features=100_000,     
    sublinear_tf=True,       
    min_df=2,                 
    analyzer='word',
    strip_accents=None,      
    lowercase=False,         
)

pipe_lr = Pipeline([
    ('tfidf', TfidfVectorizer(**tfidf_params)),
    ('clf',   LogisticRegression(
        C=5.0,
        max_iter=1000,
        solver='lbfgs',
        multi_class='multinomial',
        class_weight='balanced',
        random_state=42,
        n_jobs=-1,
    ))
])

pipe_lr.fit(X_tr, y_tr)
lr_f1 = f1_score(y_val, pipe_lr.predict(X_val), average='weighted')
print(f'Logistic Regression Val F1: {lr_f1:.4f} ({lr_f1*100:.2f}%)')

Logistic Regression Val F1: 0.8171 (81.71%)


In [10]:
pipe_sgd = Pipeline([
    ('tfidf', TfidfVectorizer(**tfidf_params)),
    ('clf',   SGDClassifier(
        loss='modified_huber',
        alpha=1e-4,
        max_iter=100,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1,
    ))
])

pipe_sgd.fit(X_tr, y_tr)
sgd_f1 = f1_score(y_val, pipe_sgd.predict(X_val), average='weighted')
print(f'SGD Classifier Val F1:      {sgd_f1:.4f} ({sgd_f1*100:.2f}%)')

SGD Classifier Val F1:      0.8257 (82.57%)


In [11]:
pipe_cnb = Pipeline([
    ('tfidf', TfidfVectorizer(**tfidf_params)),
    ('clf',   ComplementNB(alpha=0.1))
])

pipe_cnb.fit(X_tr, y_tr)
cnb_f1 = f1_score(y_val, pipe_cnb.predict(X_val), average='weighted')
print(f'Complement NB Val F1:       {cnb_f1:.4f} ({cnb_f1*100:.2f}%)')

print()
print('=== SUMMARY ===')
print(f'Logistic Regression: {lr_f1*100:.2f}%')
print(f'SGD Classifier:      {sgd_f1*100:.2f}%')
print(f'Complement NB:       {cnb_f1*100:.2f}%')

Complement NB Val F1:       0.7787 (77.87%)

=== SUMMARY ===
Logistic Regression: 81.71%
SGD Classifier:      82.57%
Complement NB:       77.87%


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
معیار ارزیابی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>F1-score</code> نام دارد و از رویکرد میانگین‌گیری وزن‌دار (<code dir=ltr>average='weighted'</code>) استفاده می‌شود.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی مجموعه‌ی آموزش یا اعتبارسنجی ارزیابی کنید و طبق نتایج به‌دست‌آمده پارامترهای مدل خود را بهتر تنظیم کنید.
</font>
</p>

In [ ]:
scores = {'Logistic Regression': (lr_f1, pipe_lr),
          'SGD Classifier': (sgd_f1, pipe_sgd),
          'Complement NB': (cnb_f1, pipe_cnb)}

best_name = max(scores, key=lambda k: scores[k][0])
best_val_f1, best_val_pipe = scores[best_name]

print(f'Best model: {best_name}')
print(f'Validation Weighted F1: {best_val_f1:.4f} ({best_val_f1*100:.2f}%)')
print()

y_pred_val = best_val_pipe.predict(X_val)

Best model: SGD Classifier
Validation Weighted F1: 0.8257 (82.57%)



<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    پس از مهندسی ویژگی و مدل‌سازی، الگوریتمی دارید که می‌تواند شما را از متغیرهای مستقل به متغیر هدف برساند.
    <br>
    از این مدل برای پیش‌بینی نمونه‌های موجود در مجموعه‌ی آزمون استفاده کنید و نتایج را در یک دیتافریم تک‌ستونه با نام <code>submission</code> و در قالب زیر آماده کنید. توجه داشته باشید که ترتیب پیش‌بینی شما اهمیت دارد یعنی به عنوان مثال پیش‌بینی مدل برای نمونه‌ی آزمون <code>m</code> ام را باید در ردیف <code>m</code> ام این دیتافریم ذخیره کنید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|<code>category</code>|پیش‌بینی مدل شما (از جنس رشته)|
    
</font>
</div>
</center>

In [14]:
pipe_sgd.fit(X_train_text, y_train)
y_pred_test_num = pipe_sgd.predict(X_test_text)
y_pred_test_str = le.inverse_transform(y_pred_test_num)
submission = pd.DataFrame({'category': y_pred_test_str})

print('Submission shape:', submission.shape)
print('\nFirst 5 rows:')
submission.head()

Submission shape: (417, 1)

First 5 rows:


,category
0,هنر و سرگرمی
1,حقوق و دولت و سیاست
2,حقوق و دولت و سیاست
3,موسیقی
4,حقوق و دولت و سیاست


In [15]:
import zipfile
import joblib
import os

if not os.path.exists(os.path.join(os.getcwd(), 'text_categorization.ipynb')):
    %notebook -e text_categorization.ipynb


def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

submission.to_csv('submission.csv', index=False)
file_names = ['text_categorization.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['text_categorization.ipynb', 'submission.csv']


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
💭 اضافه: ابرِ کلمات (Word Cloud)
</font>
</h2>

<center>
<img src="wordcloud.png">
</center>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
یکی از کتابخانه‌های بسیار جالب مرتبط با متن در پایتون، <a href="https://github.com/amueller/word_cloud"><code>WordCloud</code></a> نام دارد. این کتابخانه به شما کمک می‌کند تا ابری از پرتکرارترین توکن‌های موجود در یک مجموعه‌متن را به شکلی زیبا به تصویر بکشید. خوشبختانه نسخه‌ی فارسی این کتابخانه نیز وجود دارد که می‌توانید از <a href="https://github.com/alihoseiny/word_cloud_fa">این لینک</a> به صفحه‌ی گیت‌هاب آن مراجعه کنید. حتی می‌توانید به‌صورت دلخواه تصویری را تعیین کنید تا نمایش نهایی توکن‌ها تداعی‌کننده‌ی تصویر موردنظر شما باشد.
</font>
</p>

In [ ]:
cloud_text = ''
for text in X_train:
    cloud_text += text + ' '

In [ ]:
from wordcloud_fa import WordCloudFa

wc = WordCloudFa(width=1200, height=800, persian_normalize=True, include_numbers=False, max_words=120, background_color='white', min_font_size=10, max_font_size=100)
word_cloud = wc.generate(cloud_text)
image = word_cloud.to_image()
image.show()
image.save('wordcloud.png')

Defaulting to user installation because normal site-packages is not writeable
  Using cached wordcloud_fa-0.1.10-py3-none-any.whl.metadata (11 kB)
  Using cached arabic_reshaper-3.0.0-py3-none-any.whl.metadata (12 kB)
  Using cached python_bidi-0.4.2-py2.py3-none-any.whl.metadata (4.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      ERROR: Can not execute `setup.py` since setuptools is not available in the build environment.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


ModuleNotFoundError: No module named 'wordcloud_fa'

<h4 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
<b>راهنمایی</b>
</font>
</h4>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    ۱. از تکنیک n-gram کمک بگیرید.
    <br>
    ۲. توازن مجموعه‌داده را بررسی کنید.
    <br>
    ۳. در پیش‌پردازش خود می‌توانید حذف حروف اضافه و اعداد، حذف کلمات توقف، نرمال‌سازی و... را آزمایش کنید.
</font>
</p>